In [2]:
!pip install tokenizers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/Colab Notebooks
%ls 

Mounted at /content/gdrive
/content/gdrive/MyDrive/Colab Notebooks
'Before August 2022'/         cwe-train.txt     nlp.ipynb
'Copy of BaselineRNN.ipynb'  'Decision Tree'/   sw-train.txt
'Copy of KayalPost.ipynb'     merges.txt        vocab.json


In [4]:
vocabulary_size = 25000

In [5]:
import os
from tokenizers.models import BPE
from tokenizers import Tokenizer
from tokenizers.decoders import ByteLevel as ByteLevelDecoder
from tokenizers.normalizers import NFC, Sequence
from tokenizers.pre_tokenizers import ByteLevel
from tokenizers.trainers import BpeTrainer

class BPE_token(object):
    def __init__(self):
        self.tokenizer = Tokenizer(BPE())
        self.tokenizer.normalizer = Sequence([
            NFC()
        ])
        self.tokenizer.pre_tokenizer = ByteLevel()
        self.tokenizer.decoder = ByteLevelDecoder()

    def bpe_train(self, paths):
        trainer = BpeTrainer(vocab_size=vocabulary_size, show_progress=True, inital_alphabet=ByteLevel.alphabet(), special_tokens=[
            "<s>",
            "<pad>",
            "</s>",
            "<unk>",
            "<mask>"
        ])
        self.tokenizer.train(paths, trainer)

    def save_tokenizer(self, location, prefix=None):
        if not os.path.exists(location):
            os.makedirs(location)
        self.tokenizer.model.save(location, prefix)

In [6]:
paths = ['cwe-train.txt']

In [7]:
tokenizer = BPE_token()
tokenizer.bpe_train(paths)
tokenizer.save_tokenizer('.')

In [8]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 11.5 MB/s 
     |████████████████████████████████| 163 kB 58.7 MB/s 


In [9]:
import tensorflow as tf
from transformers import GPT2Config, TFGPT2LMHeadModel, GPT2Tokenizer
# loading tokenizer from the saved model path
tokenizer = GPT2Tokenizer.from_pretrained('.')
tokenizer.add_special_tokens({
  "eos_token": "</s>",
  "bos_token": "<s>",
  "unk_token": "<unk>",
  "pad_token": "<pad>",
  "mask_token": "<mask>"
})
# creating the configurations from which the model can be made
config = GPT2Config(
  vocab_size=tokenizer.vocab_size,
  bos_token_id=tokenizer.bos_token_id,
  eos_token_id=tokenizer.eos_token_id,
  n_embd=256,
  n_layer=4,
  n_head=4
)
# creating the model
model = TFGPT2LMHeadModel(config)
model(model.dummy_inputs)
model.summary()

Model: "tfgpt2lm_head_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (TFGPT2MainLaye  multiple                 7654656   
 r)                                                              
                                                                 
Total params: 7,654,656
Trainable params: 7,654,656
Non-trainable params: 0
_________________________________________________________________


In [10]:
single_string = ''
for filename in paths:
  with open(filename, "r", encoding='utf-8') as f:
    x = f.read()
  single_string += x + tokenizer.eos_token
string_tokenized = tokenizer.encode(single_string)

In [11]:
string_tokenized[:10]

[118, 744, 685, 164, 165, 107, 7166, 9279, 7, 8466]

In [12]:
examples = []
block_size = 100
BATCH_SIZE = 12
BUFFER_SIZE = 1000
for i in range(0, len(string_tokenized) - block_size + 1, block_size):
  examples.append(string_tokenized[i:i + block_size])
inputs, labels = [], []
for ex in examples:
  inputs.append(ex[:-1])
  labels.append(ex[1:])
dataset = tf.data.Dataset.from_tensor_slices((inputs, labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [13]:
# defining our optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
# definining our loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# defining our metric which we want to observe
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
# compiling the model
model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer], metrics=[metric])

In [14]:
num_epoch = 15
history = model.fit(dataset, epochs=num_epoch)

Epoch 1/15
86/86 [==============================] - 113s 1s/step - loss: 9.0570 - accuracy: 0.0624
Epoch 2/15
86/86 [==============================] - 100s 1s/step - loss: 8.3705 - accuracy: 0.0681
Epoch 3/15
86/86 [==============================] - 98s 1s/step - loss: 7.7751 - accuracy: 0.0717
Epoch 4/15
86/86 [==============================] - 98s 1s/step - loss: 7.2733 - accuracy: 0.0749
Epoch 5/15
86/86 [==============================] - 96s 1s/step - loss: 6.8882 - accuracy: 0.0830
Epoch 6/15
86/86 [==============================] - 98s 1s/step - loss: 6.6162 - accuracy: 0.0962
Epoch 7/15
86/86 [==============================] - 97s 1s/step - loss: 6.4387 - accuracy: 0.1100
Epoch 8/15
86/86 [==============================] - 97s 1s/step - loss: 6.3095 - accuracy: 0.1288
Epoch 9/15
86/86 [==============================] - 98s 1s/step - loss: 6.2029 - accuracy: 0.1464
Epoch 10/15
86/86 [==============================] - 97s 1s/step - loss: 6.1065 - accuracy: 0.1576
Epoch 11/15
86/86

In [15]:
text = "An bhfuil"
# encoding the input text
input_ids = tokenizer.encode(text, return_tensors='tf')
# getting out output
beam_output = model.generate(
  input_ids,
  max_length = 50,
  num_beams = 5,
  temperature = 0.4,
  no_repeat_ngram_size=2,
  num_return_sequences=5
)
print(tokenizer.decode(beam_output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 2 (first `eos_token_id`) to generate sequence


<unk>n bhfuil na wanhu wa mulungu. maabaho yesu, mbali wanhu wose, na mulungu, "vino munhu yoyose ya mulungu kwa ichimu cha chilisito. mbali yesu kawalongela, kwaviya yesu chilisito, ivo yesu. kwaviya wanhu weli na mweye


In [16]:
from transformers import WEIGHTS_NAME, CONFIG_NAME
model_to_save = model.module if hasattr(model, 'module') else model
output_model_file = os.path.join('.', WEIGHTS_NAME)
output_config_file = os.path.join('.', CONFIG_NAME)
# save model and model configs
model.save_pretrained('.')
model_to_save.config.to_json_file(output_config_file)
# save tokenizer
tokenizer.save_pretrained('.')

('./tokenizer_config.json',
 './special_tokens_map.json',
 './vocab.json',
 './merges.txt',
 './added_tokens.json')

In [17]:
# can then avoid retraining by loading these files when they exist:
tokenizer = GPT2Tokenizer.from_pretrained(output_dir)
model = TFGPT2LMHeadModel.from_pretrained(output_dir)

NameError: ignored